In [9]:
import sys
from os.path import join
import pandas as pd 
import torch

sys.path.append('../src')
from models.lib.neural import *
from models.lib.data import *
from models.lib.lightning_train import generate_trainer
from bigcsv import BigCSV



In [3]:
pd.read_csv('../data/raw/allen_m1_region.tsv', sep='\t', nrows=3)

,gene,AAACCCAAGGATTTCC-LKTX_190129_01_A01,AAACCCAAGTATGGCG-LKTX_190129_01_A01,AAACCCACAAAGTGTA-LKTX_190129_01_A01,AAACCCACACTACTTT-LKTX_190129_01_A01,AAACCCACAGTGAGCA-LKTX_190129_01_A01,AAACCCAGTCACCCTT-LKTX_190129_01_A01,AAACCCAGTGTCCACG-LKTX_190129_01_A01,AAACCCATCATAAGGA-LKTX_190129_01_A01,AAACCCATCTGTCCCA-LKTX_190129_01_A01,...,TTTGGTTGTAGTTCCA-LKTX_190130_01_H01,TTTGGTTGTTCGGTCG-LKTX_190130_01_H01,TTTGGTTTCCTTATGT-LKTX_190130_01_H01,TTTGGTTTCGGTATGT-LKTX_190130_01_H01,TTTGGTTTCGGTCGGT-LKTX_190130_01_H01,TTTGTTGAGATGGCGT-LKTX_190130_01_H01,TTTGTTGCACAGCCAC-LKTX_190130_01_H01,TTTGTTGCAGAGACTG-LKTX_190130_01_H01,TTTGTTGCATAATGAG-LKTX_190130_01_H01,TTTGTTGTCTACTCAT-LKTX_190130_01_H01
0,DDX11L1|DDX11L1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,WASH7P|WASH7P,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,MIR6859-1|MIR6859-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
converter = BigCSV(
    file='../data/raw/allen_m1_region.tsv',
    outfile='../data/raw/allen_m1_region.h5ad',
    insep='\t',
    chunksize=50,
)

converter.to_h5ad(
    index_col='gene',
    outfile='../data/raw/allen_m1_region.h5ad',
)

Chunkfolder not passed, generating...


TypeError: experimental_to_h5ad() got an unexpected keyword argument 'chunkfolder'

In [2]:
datafiles = ['../data/interim/whole_brain_bhaduri_T.csv']
labelfiles = ['../data/processed/labels/whole_brain_bhaduri_labels.csv']

In [3]:
subset = pd.read_csv(labelfiles[0])
subset = subset[subset['Type'].isin([0, 1])]
subset = subset.iloc[0:500, :]
subset.to_csv('../data/processed/labels/whole_brain_bhaduri_labels_subset.csv', index=False)

pd.read_csv('../data/processed/labels/whole_brain_bhaduri_labels_subset.csv')

,cell,Type
0,296,0
1,7277,0
2,8643,0
3,8665,0
4,8696,0
...,...,...
495,79881,0
496,79884,0
497,79889,0
498,79896,0


In [4]:
labelfiles = ['../data/processed/labels/whole_brain_bhaduri_labels_subset.csv']

trainer, model, module = generate_trainer(
    datafiles=datafiles,
    labelfiles=labelfiles,
    class_label='Type',
    index_col='cell',
    skip=3,
    shuffle=True,
    drop_last=True,
    normalize=True,
    batch_size=4,
    num_workers=0,
    weighted_metrics=True,
    weights=total_class_weights(labelfiles, 'Type'),
    optim_params={
        'optimizer': torch.optim.Adam,
        'lr': 0.02,
        'weight_decay': 3e-4,
    }
)

Device is cpu
../data/interim/whole_brain_bhaduri_T.csv exists, continuing...
../data/processed/labels/whole_brain_bhaduri_labels_subset.csv exists, continuing...



GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Model initialized. input_dim = 30741, output_dim = 2. Metrics are dict_keys(['accuracy', 'precision', 'recall']) and weighted_metrics = True
self.weights = tensor([0.6649, 2.0161])


In [5]:
trainer.fit(model, datamodule=module)

Creating train/val/test DataLoaders...
Done, continuing to training.


wandb: Currently logged in as: jlehrer1 (use `wandb login --relogin` to force relogin)



  | Name       | Type                 | Params
----------------------------------------------------
0 | base_model | TabNetGeneClassifier | 2.0 M 
----------------------------------------------------
2.0 M     Trainable params
0         Non-trainable params
2.0 M     Total params
8.138     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:453: UserWarning: Your `val_dataloader` has `shuffle=True`,it is strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of 

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]